# Documentación de código Base 

__Este documento es creado para la comprensión de cada sección de código y posteriormente su depuración__

__Creado por: Juan David Parra Cantor__

__Nota Importante__: En la primera ejecución se presenta un error en la línea 170, Por una advertencia de Sintaxys 

__170: SyntaxWarning: invalid escape sequence '\s'__
    __plt.ylabel('$\sigma_r$ [Pa]')__

Previo a está primera ejecución se revisa el código


# Parte 1

__Librerias__
- numpy: para arreglos númericos
- matplotlib: para graficos
- scipy: para manejo de arreglos matriciales y soluciones



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_banded

# Parte 2

__valores constantes__

- radio ext(Ro): 0.02 m = 2.0e-2 m
- radio inter(Ri): $$Ro - (3.0e-3)$$
    3.0e-3 = 3x10^(-3) = 0.003 m
- Concentración inicial de Hidrogeno en radio interno = 0.1
- Concentración inicial de Hidrogeno en radio externo = 0.0
- Coeficiente de difusión(D) = 1.0e-8 [m^2/s]
- Modulo de Young(E) = 10e9 [Pa]
- Coeficiente de Poisson(nu) = 0.3
- Concentración inicial de Hidrogeno en el centro(C0) = 0
- Presión Interior(pin) = 400.0e3 [Pa]
- Cantidad de nodos(n) = 2000
- Distancia entre nodos(dr): $$dr = (Ro - Ri) / (n - 1)$$
- segundos por hora(s) = 3600, __¿Cuál es la necesidad de poner el 60*60?
- Tiempo de simluación(t_end) = 5 hrs
- Paso del tiempo(nt) = 1000
- diferencial de tiempo(dt): $$dt = t_end / (nt - 1)$$



In [ ]:
Ro=2.0e-2
Ri=Ro-3.0e-3
Cin=0.1
Cout=0.0
D=1.0e-8
E=10e9 #Pa
nu=0.3
Omega=5.0e-3
C0=0.0
pin=400.0e3 #Pa https://www.ugc.edu.co/pages/juridica/documentos/institucionales/NTC_2505_Instalaciones_Suministro_De_Gas.pdf
n=2000 #nodes
dr=(Ro-Ri)/(n-1)
#dt=0.083
#S=31536000 # seconds in a year
S=60*60 # seconds per hour
t_end=5.0
#nt=int(t_end/dt)
nt=1000
dt=t_end/(nt-1)

# Parte 3

__Creación de Matrices y vectores__

- Vectores:
    - *__cold__* almacena la concentracioón en cada nodo 
    - *__c__* almacena la concentración por nodo en paso de tiempo 
    - *__Disp__* almacena la deformación en cada nodo por paso de tiempo 
    - *__rhs__* alamacena término de la derecha de la ecuación de difusión
    - *__rhsDisp__* almacena terminos de la derecha de la ecuación de deformación
    - *__r__* Almacena coordenadas radiales para cada nodo
    - *__t__* almacena los tiempos de simulación
    - *__H, Hflux, HDisp, HStress_r, HStress_t, HStrain_r, y HStrain_t__* Almacenan los resultados de la simulación en cada nodo según el paso del tiempo

- Matrices:
    - *__sigma_r__* almacena la tención radial
    - *__sigma_t__* almacena la tensión tangencial
    - *__epsi_r__* almacena la deformación radial
    - *__epsi_t__* almacena la deformación tangencial
    - *__A__* almacena coeficiente de la ecuación de difusión
    - *__ADisp__* almacena coeficiente de la ecuación de deformación



In [ ]:

Cold=np.zeros(n)
C=np.zeros(n)
Disp=np.zeros(n)
sigma_r=np.zeros(n)
sigma_t=np.zeros(n)
epsi_r=np.zeros(n)
epsi_t=np.zeros(n)
A=np.zeros((3,n))
ADisp=np.zeros((5,n))
rhs=np.zeros(n)
rhsDisp=np.zeros(n)
r=np.linspace(Ri,Ro,n)
t=np.linspace(0,nt*dt,nt+1)
H=np.zeros((n,nt+1))
Cflux=np.zeros(n)
Hflux=np.zeros((n,nt+1))
HDisp=np.zeros((n,nt+1))
HStress_r=np.zeros((n,nt+1))
HStress_t=np.zeros((n,nt+1))
HStrain_r=np.zeros((n,nt+1))
HStrain_t=np.zeros((n,nt+1))

# Parte 4

__Modelo de difusión__

